In [3]:
# Install packages
!pip -q install langchain langchain-core langchain-community pydantic langchain-experimental duckduckgo_search -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 50.5 MB/s eta 0:00:00


**Built-in Tool - DuckDuckGo Search**

In [7]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

search_tool.invoke("latest news in India?")

'Mar 29, 2025 — Breaking News in India : Read Latest News on Sports, Business, Entertainment, Blogs and Opinions from leading columnists. Times of India brings the Breaking ... 2 hours ago — Greater Noida Dowry Death: 4 Family Members, Including Husband Vipin ... · Six Killed, 86 Injured As Israeli Missiles Rain Down On Sanaa · India Warns Pakistan of ... 12 hours ago — Watch: Raging rivers cause destruction in parts of northern India . At least 30 people have died since Tuesday, as flash floods swept away roads and submerged ... Jul 4, 2025 — Latest news · Trump threatens India with 50% tariff as negotiations fizzle and Modi keeps importing Russian oil · Wall of water slams Himalayan village, leaving ... 13 hours ago — India News Live Updates | 5 stories you must read today, August 28: From Russian drone strike in Ukraine to Onam row in Kerala school and more. Read about the ...'

In [8]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


**Built-in Tool - Shell Tool**

In [9]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('ls')

print(results)

Executing command:
 ls
sample_data



/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


**Custom Tools**

In [10]:
from langchain_community.tools import tool

In [11]:
# Step 1: Create function

def multiply(a,b):
  """Mutiply two numbers"""
  return a*b

In [12]:
# Step 2: Add type hints
def multiply (a:int, b:int) -> int:
  """Mutiply two numbers"""
  return a*b

In [13]:
# Step 3: Add tool decorator
@tool
def multiply(a:int, b:int) -> int:
  """Mutiply two numbers"""
  return a*b

In [14]:
results = multiply.invoke({"a":50, "b":50})
print(results)

2500


In [15]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Mutiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [16]:
print(multiply.args_schema.model_json_schema())

{'description': 'Mutiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


**Method 2 - Using StructuredTool**

In [17]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [18]:
 class MultiplyInput(BaseModel):
  a: int = Field(required = True, description = "The first number to add")
  b: int = Field(required = True, description = "The second number to add")

In [19]:
def multiply_func(a:int, b:int)-> int:
  return a*b

In [20]:
multiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name = "multiply",
    description = "Multiply two numbers",
    args_schema = MultiplyInput
)

In [21]:
result =multiply_tool.invoke({'a':450, "b":297})

In [22]:
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

133650
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


**Method 3 - Using BaseTool Class**

In [23]:
from langchain.tools import BaseTool
from typing import Type

In [24]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
  a: int = Field(required=True, description="The first number to add")
  b: int = Field(required=True, description="The second number to add")

In [25]:
class MultiplyTool(BaseTool):
  name: str = "multiply"
  description: str = "Multiply two numbers"
  args_schema: Type[BaseModel] = MultiplyInput

  def _run(self, a:int, b:int) ->int:
    return a * b

In [26]:
multiply_tool = MultiplyTool()

In [27]:
result = multiply_tool.invoke({'a':450, "b":297})

In [28]:
print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

133650
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


**Toolkit**

In [29]:
from langchain_core.tools import tool

#Custom tools

@tool
def add (a:int, b:int) -> int:
  """Add two numbers"""
  return a + b

@tool
def multiply (a:int, b:int) -> int:
  """Multiply two numbers"""
  return a * b


In [30]:
class MathToolKit:
  def get_tools(self):
    return[add, multiply]

In [31]:
toolkit = MathToolKit()
tools = toolkit.get_tools()

for tool in tools:
  print(tool.name,"=>", tool.description)

add => Add two numbers
multiply => Multiply two numbers
